In [1]:
# 1. Importamos todas las librerías que creemos útiles

import io # Provee diferentes tipos de E/S: texto E/S, binario E/S e E/S.

import os # Leer o escribir un archivo.

import math # Proporciona acceso a las funciones matemáticas definidas en C.

import csv # Para cargar y manejar CSV.

import xlrd # Funcionalidades de excel en Python (1/2).

import xlwt # Funcionalidades de excel en Python (2/2).

import sklearn # Funcionalidades de Machine Learning.

import numpy as np # Manejo de datos extremadamente rápido cálculo numérico.

import scipy as sp # Módulos para optimización, álgebra lineal, etc.

import pandas as pd # Módulos de gestión de ficheros y dataframes.

import matplotlib.pyplot as plt # Generación de gráficos.

import statsmodels.api as sm # Módulo estadístico.

import random # Para fijar resultados.

In [2]:
# 2.1 Leo el fichero6.

fichero6 = pd.read_csv(r'C:\Users\jlc15\Desktop\MECOFIN\2º Cuatrimestre\TFM\Y - Ficheros Python\fichero6.csv')

fichero6

,Gender,Education,Work Exp,Programming Exp,Matching,Status
0,0,1,0,0,0,0
1,1,1,0,1,1,1
2,1,1,1,0,1,1
3,1,1,0,0,0,0
4,1,0,1,0,0,1
...,...,...,...,...,...,...
824,1,1,0,0,1,1
825,1,1,1,0,1,1
826,1,1,0,1,0,0
827,1,1,0,0,1,1


In [3]:
# 3.1 Curva ROC (con Status y Matching).

from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, thresholds = roc_curve(fichero6['Status'], fichero6['Matching'])

auc = roc_auc_score(fichero6['Status'], fichero6['Matching'])

auc

0.9271929824561403

In [4]:
# 3.2 Conclusión:

# El valor AUC es el coeficiente que expone la zona de predicción bajo la
# curva ROC. Cuanto más alto sea el coeficiente, mejor calidad de predicción.
# El intervalo oscila entre 0 y 1.

In [5]:
# 4.1 Calculamos el coeficiente Kappa.

from sklearn.metrics import cohen_kappa_score

kappa = cohen_kappa_score(fichero6['Status'], fichero6['Matching'])

kappa

0.7856967107168775

In [6]:
# 4.2 Conclusión:

# El coeficiente Kappa al ser superior a 0.5 muestra que no abundan los 
# eventos aleatorios en el dataset al evaluar los 2 clasificadores.

In [7]:
# 5.1 Calculamos los coeficientes de correlación de Pearson.

corr = fichero6.corr()

corr

,Gender,Education,Work Exp,Programming Exp,Matching,Status
Gender,1.000000,-0.060435,0.129666,-0.204122,0.023318,0.043740
Education,-0.060435,1.000000,-0.043051,0.017417,0.523623,0.086997
Work Exp,0.129666,-0.043051,1.000000,-0.234425,0.041665,0.071721
Programming Exp,-0.204122,0.017417,-0.234425,1.000000,-0.045861,-0.066631
Matching,0.023318,0.523623,0.041665,-0.045861,1.000000,0.804385
Status,0.043740,0.086997,0.071721,-0.066631,0.804385,1.000000


In [8]:
# 5.2 Conclusiones:

# a) El coeficiente revela que las variables explicativas no están 
#    correlacionadas. Es más, rondan entre -0.2 y 0.2, una muy buena señal.

# b) Se puede comprobar que la variable educación determina en gran parte
#    el valor de Matching. Esto es completamente cierto, ya que Assurance 
#    está construido con las variables expuestas excepto género.

# c) Como era de esperar, Matching es la que más correlación posee con Status
#    aunque de una forma bastante leve (no llega a 1).
#    Es importante tener en cuenta que un valor cercano a cero no necesariamente
#    significa que no haya relación entre las dos variables, ya que puede haber
#    una relación no lineal entre ellas que no se pueda capturar con el coeficiente 
#    de correlación de Pearson.

In [16]:
# 6.1 Distancia del coseno:

from sklearn.metrics.pairwise import cosine_distances

vector1 = fichero6.iloc[0, :-1].values
vector2 = fichero6.iloc[1, :-1].values

distancia = cosine_distances([vector1], [vector2])[0, 0]

print(f"Distancia del coseno: {distancia}")

Distancia del coseno: 0.5


In [ ]:
# 6.2 Conclusiones:

# a) Al ser una distancia de 0.5, los vectores coinciden en un 50%, lo que significa
# que hay un nivel intermedio de similitud entre el vector 1 y el 2.

# b) Es decir, si un resultado es cercano a 1 los datos indicaria una similitid perfecta,
# un reultado -1 una similitud totalmente inversa, y un resultado de 0, una ausencia de similitud.
# Por ello, un resultado de 0.5 se puede considerar un resultado razonablemente bueno en lo
# que respecta a la calidad de los datos y la relación que poseen entre sí.

In [ ]:
# 7.1 Calculamos la tolerancia o VIF inverso (o tolerancia).

# Si incluimos Matching como es lógico, se da multicolinealidad.

x = fichero6[['Gender', 'Education', 'Work Exp', 'Programming Exp']]
y = fichero6['Status']

x = sm.add_constant(x)

modelo = sm.Logit(y, x)
resultado = modelo.fit()

vif = 1 / (1 - resultado.pvalues[1:]) 
tolerancia = vif.round(3)

print("Tolerancia:")
print(tolerancia)

In [ ]:
# 7.2 Conclusiones:

# a) Ninguna de las variable del modelo da señales de colinealidad.

# b) No se incluye la vble Matching puesto que ya sabemos con Pearson
#    que la correlación es alta entre esta, Status y Educación. Por lo que,
#    lo que nos interesa más es saber si el modelo original tiene problemas
#    de multicolinealidad. En caso positivo tendríamos que rehacer el modelo.

In [ ]:
# 8.1 Calculamos el recall.

from sklearn.metrics import recall_score

recall = recall_score(fichero6['Status'], fichero6['Matching'])

recall

In [ ]:
# 8.2 Conclusión:

# El valor de recall nos da 0.85. Esto significa que Matching predice 
# en un 85.43% los valores de Status. Es una muy buena señal.

In [ ]:
# 9.1 Calculamos su F1 con y sin Matching

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

x = fichero6[['Gender', 'Education', 'Work Exp', 'Programming Exp', 'Matching']]
y = fichero6['Status']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

modelo = LogisticRegression()
modelo.fit(x_train, y_train)

y_pred = modelo.predict(x_test)

f1 = f1_score(y_test, y_pred)

print("La medida F1 es: {:.2f}".format(f1))

In [ ]:
# 9.2 Conclusiones:

# a) Sin Matching nos da un resultado de 0.78 y son Matching un resultado
#    de 0.93. Esto refleja una buena precisión y exhaustividad en el modelo.

# b) También refleja un mejor comportamiento incluyendo la variable auxiliar.